# Sequence data

- 시퀀스 데이터는 순서가 있고 이것이 중요한 데이터로
- 자연어, 유전, 시계열 자료 등이 이런 자료형에 속한다.
  
    
- 모델링시 중요한 점은 이 순서의 의미를 반영해주어야 한다는 점이고
- 가장 각광받는 RNN과 CNN에 대해서 소개한다.

## RNN 

- 재귀적/순한 신경망을 뜻하며 데이터를 다룰 시에 히든 노드가 다음 히든 노드에 영향을 주는 모형
- 마찬가지로 역전파 알고리즘을 이용하여 업데이트하나 생각보다 코드 구현이 쉽지 않음(BPTT)
- 심플한 RNN보다는 LSTM이 엄청난 각광을 받았고 한국교수가 제안한 GRU도 유명함
- 코드 구성시 얼만큼의 시간을 고려할지를 인풋으로 넣어줘야함
    - function만들어서 가능
    - 자료형을 변형시켜서 가능
    

## CNN

- 본래 이미지를 대상으로하는 분석에서 엄청난 획을 그은 분석 기법
- 이미지관련 분석은 CNN전과 후로 나눌 수 있을 듯
- 시퀀스데이터에서도 비슷한 개념을 활요하여 접근 가능

In [1]:
from sklearn.datasets import load_boston
import numpy as np
import keras
from keras import models, layers
from keras.callbacks import EarlyStopping
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

d = load_boston()

Using TensorFlow backend.


In [2]:
d.data[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [3]:
temp_data = (d.data-np.mean(d.data))/np.std(d.data)

In [4]:
data = []
lags = 15

for i in range(lags, len(temp_data)):
    temp = temp_data[max(0, i-lags):i]
    data.append(temp)
    
data = np.array(data)

In [5]:
data.shape

(491, 15, 13)

In [17]:
d.target

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
       18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
       15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
       13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
       21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
       35.4, 24.7, 31.6, 23.3, 19.6, 18.7, 16. , 22.2, 25. , 33. , 23.5,
       19.4, 22. , 17.4, 20.9, 24.2, 21.7, 22.8, 23.4, 24.1, 21.4, 20. ,
       20.8, 21.2, 20.3, 28. , 23.9, 24.8, 22.9, 23.9, 26.6, 22.5, 22.2,
       23.6, 28.7, 22.6, 22. , 22.9, 25. , 20.6, 28.4, 21.4, 38.7, 43.8,
       33.2, 27.5, 26.5, 18.6, 19.3, 20.1, 19.5, 19.5, 20.4, 19.8, 19.4,
       21.7, 22.8, 18.8, 18.7, 18.5, 18.3, 21.2, 19.2, 20.4, 19.3, 22. ,
       20.3, 20.5, 17.3, 18.8, 21.4, 15.7, 16.2, 18. , 14.3, 19.2, 19.6,
       23. , 18.4, 15.6, 18.1, 17.4, 17.1, 13.3, 17.8, 14. , 14.4, 13.4,
       15.6, 11.8, 13.8, 15.6, 14.6, 17.8, 15.4, 21

In [44]:
keras.backend.clear_session()

input_layer = layers.Input(batch_shape=data.shape)

l1 = layers.Bidirectional(layers.GRU(128, return_sequences=True, stateful=True))(input_layer)

l2 = layers.Bidirectional(layers.GRU(64, return_sequences=False, stateful=True))(l1)

l3 = layers.Dense(16, activation='relu')(l2)

out_layer = layers.Dense(1, activation='linear')(l3)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

es = EarlyStopping(patience=20, monitor='val_loss', restore_best_weights=True)

In [45]:
model.fit(data, d.target[lags:],
         epochs=400,
#          validation_split=0.2,
          batch_size=len(data),
         callbacks=[es]
         )

Epoch 1/400
491/491 [==============================] - 2s 3ms/step - loss: 595.6014 - mae: 22.5913
Epoch 2/400
491/491 [==============================] - 0s 487us/step - loss: 564.6003 - mae: 21.8684
Epoch 3/400
491/491 [==============================] - 0s 481us/step - loss: 545.9798 - mae: 21.4275
Epoch 4/400
491/491 [==============================] - 0s 496us/step - loss: 522.8373 - mae: 20.8707
Epoch 5/400
491/491 [==============================] - 0s 508us/step - loss: 498.5291 - mae: 20.2730
Epoch 6/400
491/491 [==============================] - 0s 674us/step - loss: 473.8902 - mae: 19.6511
Epoch 7/400
491/491 [==============================] - 0s 705us/step - loss: 449.2385 - mae: 19.0019
Epoch 8/400
491/491 [==============================] - 0s 707us/step - loss: 422.3438 - mae: 18.2771
Epoch 9/400
491/491 [==============================] - 0s 678us/step - loss: 394.7408 - mae: 17.5106
Epoch 10/400
491/491 [==============================] - 0s 664us/step - loss: 367.4346 - mae:

Epoch 83/400
491/491 [==============================] - 0s 723us/step - loss: 85.8668 - mae: 6.7771
Epoch 84/400
491/491 [==============================] - 0s 758us/step - loss: 85.8625 - mae: 6.7760
Epoch 85/400
491/491 [==============================] - 0s 727us/step - loss: 85.8558 - mae: 6.7743
Epoch 86/400
491/491 [==============================] - 0s 733us/step - loss: 85.8470 - mae: 6.7719
Epoch 87/400
491/491 [==============================] - 0s 743us/step - loss: 85.8366 - mae: 6.7691
Epoch 88/400
491/491 [==============================] - 0s 725us/step - loss: 85.8249 - mae: 6.7658
Epoch 89/400
491/491 [==============================] - 0s 727us/step - loss: 85.8124 - mae: 6.7624
Epoch 90/400
491/491 [==============================] - 0s 715us/step - loss: 85.7992 - mae: 6.7586
Epoch 91/400
491/491 [==============================] - 0s 733us/step - loss: 85.7858 - mae: 6.7546
Epoch 92/400
491/491 [==============================] - 0s 760us/step - loss: 85.7725 - mae: 6.7504


Epoch 165/400
491/491 [==============================] - 0s 725us/step - loss: 85.6558 - mae: 6.6848
Epoch 166/400
491/491 [==============================] - 0s 747us/step - loss: 85.6558 - mae: 6.6847
Epoch 167/400
491/491 [==============================] - 0s 787us/step - loss: 85.6558 - mae: 6.6847
Epoch 168/400
491/491 [==============================] - 0s 766us/step - loss: 85.6558 - mae: 6.6846
Epoch 169/400
491/491 [==============================] - 0s 764us/step - loss: 85.6558 - mae: 6.6846
Epoch 170/400
491/491 [==============================] - 0s 743us/step - loss: 85.6558 - mae: 6.6845
Epoch 171/400
491/491 [==============================] - 0s 790us/step - loss: 85.6558 - mae: 6.6845
Epoch 172/400
491/491 [==============================] - 0s 784us/step - loss: 85.6558 - mae: 6.6844
Epoch 173/400
491/491 [==============================] - 0s 735us/step - loss: 85.6558 - mae: 6.6843
Epoch 174/400
491/491 [==============================] - 0s 745us/step - loss: 85.6558 - ma

491/491 [==============================] - 0s 747us/step - loss: 85.6556 - mae: 6.6836
Epoch 247/400
491/491 [==============================] - 0s 762us/step - loss: 85.6556 - mae: 6.6836
Epoch 248/400
491/491 [==============================] - 0s 758us/step - loss: 85.6556 - mae: 6.6836
Epoch 249/400
491/491 [==============================] - 0s 741us/step - loss: 85.6556 - mae: 6.6836
Epoch 250/400
491/491 [==============================] - 0s 743us/step - loss: 85.6556 - mae: 6.6835
Epoch 251/400
491/491 [==============================] - 0s 790us/step - loss: 85.6556 - mae: 6.6835
Epoch 252/400
491/491 [==============================] - 0s 750us/step - loss: 85.6556 - mae: 6.6835
Epoch 253/400
491/491 [==============================] - 0s 739us/step - loss: 85.6556 - mae: 6.6835
Epoch 254/400
491/491 [==============================] - 0s 741us/step - loss: 85.6556 - mae: 6.6835
Epoch 255/400
491/491 [==============================] - 0s 768us/step - loss: 85.6556 - mae: 6.6835
Epoc

Epoch 328/400
491/491 [==============================] - 0s 764us/step - loss: 85.6553 - mae: 6.6835
Epoch 329/400
491/491 [==============================] - 0s 764us/step - loss: 85.6554 - mae: 6.6835
Epoch 330/400
491/491 [==============================] - 0s 758us/step - loss: 85.6554 - mae: 6.6835
Epoch 331/400
491/491 [==============================] - 0s 761us/step - loss: 85.6554 - mae: 6.6835
Epoch 332/400
491/491 [==============================] - 0s 752us/step - loss: 85.6554 - mae: 6.6835
Epoch 333/400
491/491 [==============================] - 0s 750us/step - loss: 85.6554 - mae: 6.6835
Epoch 334/400
491/491 [==============================] - 0s 756us/step - loss: 85.6553 - mae: 6.6835
Epoch 335/400
491/491 [==============================] - 0s 756us/step - loss: 85.6554 - mae: 6.6835
Epoch 336/400
491/491 [==============================] - 0s 733us/step - loss: 85.6553 - mae: 6.6835
Epoch 337/400
491/491 [==============================] - 0s 772us/step - loss: 85.6554 - ma

In [20]:
model.evaluate(data, d.target[lags:], batch_size=len(data))

491/491 [==============================] - 0s 279us/step


[61.184791564941406, 5.592440128326416]

In [46]:
model.evaluate(data, d.target[lags:], batch_size=len(data))

491/491 [==============================] - 0s 551us/step


[85.65460205078125, 6.683468818664551]

In [ ]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

x1 = layers.Conv1D(filters=32, kernel_size=3, padding='same')(input_layer)
x1 = layers.MaxPooling1D()(x1)

x2 = layers.Conv1D(filters=16, kernel_size=5, padding='same')(input_layer)
x2 = layers.MaxPooling1D()(x2)

x = layers.concatenate([x1, x2])
x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(32, activation='relu')(x)

out_layer = layers.Dense(1, activation='linear')(x)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [ ]:
data2 = np.reshape(data, (496, 10, 13))

In [ ]:
model.fit(data2, d.target[lags:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es]
         )

In [ ]:
model.evaluate(data, d.target[lags:])

In [ ]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

x = layers.Conv1D(filters=32, kernel_size=3)(input_layer)
x = layers.AveragePooling1D()(x)
x = layers.Conv1D(filters=16, kernel_size=3)(x)
x = layers.AveragePooling1D()(x)
# x = layers.GlobalAveragePooling1D()(x)

x = layers.Bidirectional(layers.GRU(16, return_sequences=False))(x)

x = layers.Dense(32, activation='relu')(x)

out_layer = layers.Dense(1, activation='linear')(x)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [ ]:
model.fit(data2, d.target[lags:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es]
         )

In [ ]:
model.evaluate(data, d.target[lags:])